In [1]:
pip install --upgrade google-cloud-vision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 714.1 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.5/120.5 kB 2.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.4/181.4 kB 3.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 3.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 13.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 6.1 MB/s eta 0:00:00
  Using cached rsa-4.9-py3-none-any.whl (34 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 2.9 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 20.9 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.2.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2

# 加载图片
img = cv2.imread('1-house.jpg', 0)

# 使用 Canny 边缘检测
edges = cv2.Canny(img, 100, 200)

# 保存边缘化处理的图片
cv2.imwrite('edge_detected_house-1.jpg', edges)

True

In [4]:
from typing import Sequence
from google.cloud import vision
import io


def analyze_floorplan(path,
    feature_types: Sequence,)-> vision.AnnotateImageResponse:
    """分析户型图"""

    # 实例化一个 Vision 客户端
    client = vision.ImageAnnotatorClient()

    # 读取图像文件
    with io.open(path, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    features = [vision.Feature(type_=feature_type) for feature_type in feature_types]
    request = vision.AnnotateImageRequest(image=image, features=features)
    # 调用 Vision API
    # response = client.document_text_detection(image=image)
    response = client.annotate_image(request=request)
    return response

def print_objects(response: vision.AnnotateImageResponse):
    print("Objects Annotations")
    print("-" * 80)
    for obj in response.localized_object_annotations:
        nvertices = obj.bounding_poly.normalized_vertices
        print(
            f"{obj.score:4.0%}",
            f"{obj.name:15}",
            f"{obj.mid:10}",
            ",".join(f"({v.x:.1f},{v.y:.1f})" for v in nvertices),
            sep=" | ",
        )
    print("\n")
    
def print_labels(response: vision.AnnotateImageResponse):
    print("Label Annotations")
    print("-" * 80)
    for label in response.label_annotations:
        print(
            f"{label.score:4.0%}",
            f"{label.description:5}",
            sep=" | ",
        )
    print("\n")
    
        
def print_text(response: vision.AnnotateImageResponse):
    print("Text Annotations")
    print("-" * 80)
    for annotation in response.text_annotations:
        vertices = [f"({v.x},{v.y})" for v in annotation.bounding_poly.vertices]
        print(
            f"{repr(annotation.description):42}",
            ",".join(vertices),
            sep=" | Vertices: ",
        )
    print("\n")
    
        
def print_faces(response: vision.AnnotateImageResponse):
    print("Face Annotations")
    print("-" * 80)
    for face_number, face in enumerate(response.face_annotations, 1):
        vertices = ",".join(f"({v.x},{v.y})" for v in face.bounding_poly.vertices)
        print(f"# Face {face_number} @ {vertices}")
        print(f"Joy:     {face.joy_likelihood.name}")
        print(f"Exposed: {face.under_exposed_likelihood.name}")
        print(f"Blurred: {face.blurred_likelihood.name}")
        print("\n")
    print("\n")
    
        
def print_landmarks(response: vision.AnnotateImageResponse, min_score: float = 0.5):
    print("Landmark Annotations")
    print("-" * 80)
    for landmark in response.landmark_annotations:
        if landmark.score < min_score:
            continue
        vertices = [f"({v.x},{v.y})" for v in landmark.bounding_poly.vertices]
        lat_lng = landmark.locations[0].lat_lng
        print(
            f"{landmark.description:18}",
            ",".join(vertices),
            f"{lat_lng.latitude:.5f}",
            f"{lat_lng.longitude:.5f}",
            sep=" | ",
        )
    print("\n")

def print_image_properties(response: vision.AnnotateImageResponse, min_score: float = 0.5):
    print("Image Properties Annotations")
    print("-" * 80)
    for color in response.image_properties_annotation.dominant_colors.colors:
        # if color.score < min_score:
        #     continue
        print(
            "RGB(",f"{color.color.red:.0f},",f"{color.color.green:.0f},",f"{color.color.blue:.0f}",f"{')':20}","|",f"Score:{color.score:4.0%}","|",f"Pixel fraction:{color.pixel_fraction:4.0%}"
        )
    print("\n")
    

In [6]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="env")
path = "126.jpeg"
features = [
    vision.Feature.Type.OBJECT_LOCALIZATION,
    vision.Feature.Type.FACE_DETECTION,
    vision.Feature.Type.LANDMARK_DETECTION,
    vision.Feature.Type.LABEL_DETECTION,
    vision.Feature.Type.TEXT_DETECTION,
    vision.Feature.Type.IMAGE_PROPERTIES,
]

response = analyze_floorplan(path, features)

print_objects(response)
print_labels(response)
print_text(response)
print_faces(response)
print_landmarks(response)
print_image_properties(response)
# print(response)

Objects Annotations
--------------------------------------------------------------------------------
 51% | Building        | /m/0cgh4   | (0.0,0.7),(0.2,0.7),(0.2,0.9),(0.0,0.9)


Label Annotations
--------------------------------------------------------------------------------
 97% | Sky  
 82% | Tree 
 81% | Building
 80% | Cloud
 79% | Facade
 78% | Symmetry
 77% | Landmark
 75% | City 
 73% | Metropolis
 72% | Arch 


Text Annotations
--------------------------------------------------------------------------------
'全图虫创意\n头条@栀子随风城市百科'                       | Vertices: (5,397),(625,397),(625,419),(5,419)
'全'                                        | Vertices: (5,411),(12,411),(12,419),(5,419)
'图'                                        | Vertices: (16,410),(23,410),(23,418),(16,418)
'虫'                                        | Vertices: (25,410),(32,410),(32,418),(25,418)
'创意'                                       | Vertices: (35,410),(53,410),(53,418),(35,418)
'头条'                  